In [22]:
import os
import json
import numpy as np
import tensorflow as tf

In [23]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [24]:
# Name of directory for grids
directory_name = './data/new_format/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_distance.append(i['distance'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()

# direct_count = [0 for i in range(4)]
# sample = []

# # iterate through all training grids
# for file_name in os.listdir(directory_name):
#     f = open(directory_name + file_name)
#     data = json.load(f)

#     # Iterate through all the data in a given grid and append their input and output values
#     for i in data:
#         if direct_count[i['direction']] < 10000:
#             sample.append(i)
#             direct_count[i['direction']] += 1
    
#     # Close file socket
#     f.close()

# np.random.shuffle(sample)
# for i in sample:
#     x_train_grid.append(i['gridworld'])
#     x_train_position.append(i['position'])
#     x_train_distance.append(i['distance'])
#     x_train_locals.append(i['local'])
#     y_train.append(i['direction'])
    

In [25]:
#print(np.reshape(x_train_grid[25], (30,30)))
#print(x_train_position[25])
#print(y_train[25])

In [26]:
print(direct_count)

[3855, 8944, 22194, 32370]


In [27]:
# Name of directory for grids
directory_name = './data/new_format_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_distance.append(i['distance'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [28]:
#print(x_train_position)
#print(y_train)

In [29]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 30, 30) )
train_in_position = np.reshape( x_train_position, (-1, 2, 1) )
train_in_distance = np.reshape( x_train_distance, (-1,1,1) ) / 60
train_in_locals = np.reshape( x_train_locals, (-1, 3, 3) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 30, 30) )
test_in_position = np.reshape( x_test_position, (-1, 2, 1) )
test_in_distance = np.reshape( x_test_distance, (-1, 1, 1) ) / 60
test_in_locals = np.reshape( x_test_locals, (-1, 3, 3) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [30]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (30,30) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [31]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [32]:
# flatten the distance inputs
distance_input = tf.keras.layers.Input( shape = (1,1) )
flatten_distance = tf.keras.layers.Flatten()( distance_input )

In [33]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (3,3) )
flatten_local = tf.keras.layers.Flatten()( local_input )

In [34]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_local, flatten_position])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 911), dtype=tf.float32, name=None), name='concatenate_1/concat:0', description="created by layer 'concatenate_1'")


In [35]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 50, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

In [36]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, local_input, position_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [37]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [38]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

26	108	0	0
64	308	0	0
139	822	0	0
262	1151	0	0


In [39]:
# Train the model
history = model.fit( [train_in_grid, train_in_locals, train_in_position], train_out, validation_data=([test_in_grid, test_in_locals, test_in_position], test_out), epochs = 5 )

Epoch 1/10
2106/2106 [==============================] - 9s 4ms/step - loss: 0.9910 - accuracy: 0.5771 - val_loss: 1.7094 - val_accuracy: 0.2208
Epoch 2/10
2106/2106 [==============================] - 7s 3ms/step - loss: 0.7762 - accuracy: 0.6896 - val_loss: 0.6223 - val_accuracy: 0.7642
Epoch 3/10
2106/2106 [==============================] - 7s 3ms/step - loss: 0.6481 - accuracy: 0.7477 - val_loss: 0.5787 - val_accuracy: 0.7674
Epoch 4/10
2106/2106 [==============================] - 7s 3ms/step - loss: 0.5459 - accuracy: 0.7877 - val_loss: 0.4264 - val_accuracy: 0.8205
Epoch 5/10
2106/2106 [==============================] - 7s 3ms/step - loss: 0.4727 - accuracy: 0.8175 - val_loss: 0.3950 - val_accuracy: 0.8444
Epoch 6/10
2106/2106 [==============================] - 7s 3ms/step - loss: 0.4405 - accuracy: 0.8323 - val_loss: 0.6662 - val_accuracy: 0.7163
Epoch 7/10
2106/2106 [==============================] - 7s 3ms/step - loss: 0.4744 - accuracy: 0.8203 - val_loss: 0.3862 - val_accuracy:

In [40]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

0	76	57	1
0	219	142	11
0	279	681	1
0	17	6	1390
